In [13]:
%pip install Tweepy

     |████████████████████████████████| 153kB 2.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Definición de Librería

In [2]:
import tweepy
import json
import pandas as pd
import time


In [3]:
pd.set_option('display.max_rows', 1000)

In [4]:
#Definción de variables
dataFollowing_id = pd.DataFrame()
dataFollowers_id = pd.DataFrame()

# Definición de las claves para acceder a Twitter

In [5]:
#Variable de autentificación en Twitter
OAUTH_TOKEN='1204477616398503936-7Z5WSIa0490S7HbIoPrDzosdA8aJKb'
OAUTH_SECRET='OCHWxZGQlrnaoNx4corTWHNZPrtGtItteYnow7nMDhtON'
CONSUMER_KEY='IRuah0diiQDx7ayuHaFmCOvTq'
CONSUMER_SECRET='FkYZE9vZXWMgWvovRjcqQaixPleh6ADXdD3XU05ZsIkscSFzXd'

In [6]:
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=3600)

In [16]:
#Búsqueda de Tweets por palabra
def busqueda_tweets_palabra(palabra="Ryanair", lenguaje="es", numero_de_Tweets=200):
    return tweepy.Cursor(api.search, palabra, lang=lenguaje).items(numero_de_Tweets)

tweets=busqueda_tweets_palabra()
outtweets = [(tweet.created_at, 
              tweet.text, 
              tweet.retweet_count, 
              str(tweet.favorite_count)+'') for tweet in tweets]
outtweets

[(datetime.datetime(2019, 12, 18, 19, 3, 53),
  'Ryanair atribuye los recortes en Girona a los problemas del Boeing 737 MAX https://t.co/asx2LZmyV3 vía @elpais_economia',
  0,
  '0'),
 (datetime.datetime(2019, 12, 18, 19, 0, 8),
  'Los asientos separados, el nuevo mercadillo de las low cost (Ryanair, Vueling, etc...) https://t.co/Zg75Rhm4re',
  0,
  '0'),
 (datetime.datetime(2019, 12, 18, 18, 49, 18),
  'RT @marcosleyley: He visto sardinas en lata que tienen más espacio que volar en Vueling o Ryanair. Es curioso que mientras la población mun…',
  3,
  '0'),
 (datetime.datetime(2019, 12, 18, 18, 48, 17),
  'si tienes cualquier problema para que puedas resolverlo "perfectamente" te ponen un número de atención al cliente d… https://t.co/e94lP2nyZ5',
  0,
  '1'),
 (datetime.datetime(2019, 12, 18, 18, 40),
  'viajar con @Ryanair_ES y otras maneras de solo tener problemas.',
  0,
  '2'),
 (datetime.datetime(2019, 12, 18, 18, 36, 2),
  'Inspección de Trabajo abrirá expediente ✈️ https://t.co/

# Extracción de los datos de la cuenta

In [8]:
item = api.get_user('@elcorteingles')
data_account=[]
account={}
account["name"]=item.name
account["id"]=item.id
account["screen name"]=item.screen_name
account["friends_count"]=item.friends_count
account["followers_count"]=item.followers_count
data_account.append(account)
df_account_info = pd.DataFrame(data_account)
df_account_info

,name,id,screen name,friends_count,followers_count
0,El Corte Inglés,262541531,elcorteingles,1630,288870


In [16]:
#Guardado en ficheros de los IDS de Twitter
def saveID(dataframe,file):
    dataframe.to_csv(file)

saveID(df_account_info,'account_info.csv')
#saveID(dataFollowers_id,'followers_id.csv')

# Extracción de las cuentas Followers

In [7]:
#Nueva forma de obtener todas las cuentas que siguen a Uber/Cabify en España
#cabify_espana
#@Uber_ES
#NokiaMobile
# INFORMACION DE LIMITACION: https://developer.twitter.com/en/docs/basics/rate-limits
def get_followers_ids(api, twitter_account='@cabify_espana'):
    while True:
        try:
            for page in tweepy.Cursor(api.followers_ids, id=twitter_account, count=5000).pages():
                followers_ids.extend(page)
                #print(len(followers_ids))
            break
        except tweepy.TweepError as e:
            time.sleep(60*15)
            continue
    return followers_ids

followers_ids=[]
get_followers_ids(api)
#Comprobación de que el número de followers es el correcto
print(len(followers_ids))


26828


In [8]:
#Guardado de los ids en los ficheros para agilizar el desarrollo
dataFollowers_id= pd.DataFrame(followers_ids)
dataFollowers_id.columns = ['id_Follower']
dataFollowers_id

,id_Follower
0,85946106
1,553027607
2,1089180826506539008
3,961304606339977217
4,83348495
...,...
26823,306348108
26824,397919847
26825,3054151
26826,356199655


# Extracción de las cuentas Followings

In [9]:
#Nueva forma de obtener todas las cuentas sigue El Corte Inglés/Cabify
#cabify_espana
#elcorteingles

# INFORMACION DE LIMITACION: https://developer.twitter.com/en/docs/basics/rate-limits
def get_following_ids(api, twitter_account='@cabify_espana'):
    while True:
        try:
            for page in tweepy.Cursor(api.friends_ids, id=twitter_account, count=5000).pages():
                following_ids.extend(page)
                #print(len(following_ids))
            break
        except tweepy.TweepError as e:
            time.sleep(60*15)
            continue
    return following_ids

following_ids=[]
get_following_ids(api)
print(len(following_ids))


632


In [10]:
#Guardado de los ids en los ficheros para agilizar el desarrollo
dataFollowing_id= pd.DataFrame(following_ids)
dataFollowing_id.columns = ['id_Following']
dataFollowing_id


,id_Following
0,714366811
1,39787443
2,194161870
3,48422383
4,1173984355800293377
5,2992924523
6,715535722086801408
7,906103502841290752
8,428136372
9,913184120100081664


# Definición de tramos de las cuentas Followers

In [15]:
#Agrupación de ID por la limitación de la API de Twitter
followers_limit=[followers_ids[n:n+99] for n in range(0, len(followers_ids), 100)]
print(len(followers_limit))

269


In [16]:
print(followers_limit)

[[85946106, 553027607, 1089180826506539008, 961304606339977217, 83348495, 845049949981216769, 380750832, 267331464, 892541297650749440, 3406112501, 415148515, 370317367, 1225348796852424706, 1169629119132385281, 191214045, 588560915, 990049779894845440, 492595512, 2428736294, 108740930, 313839540, 1105167868621139968, 601645697, 384923310, 1224692433734205440, 3216644049, 860100474715951104, 617405147, 534771080, 163434671, 1224369471256768512, 1224454290590269440, 1528923511, 1224500733212536832, 735467174819696642, 1219916472086077440, 721969343656013824, 958378352, 789074732687720448, 1221621728184623104, 619239422, 215002897, 563473428, 1223542267660644352, 221867587, 4898496711, 317555938, 1266795414, 142864449, 1108186707399270400, 1076851198408491008, 3082680657, 1093095183485468673, 773175265891581952, 2854997973, 133433308, 1006229254537990145, 1164276844398555136, 899409043927773184, 1222202681579724800, 707017046, 1222481132576485378, 978689133009887232, 1212819020241522688,

# Definición de tramos de las cuentas Following

In [11]:
#Agrupación de ID por la limitación de la API de Twitter
following_limit=[following_ids[n:n+99] for n in range(0, len(following_ids), 100)]
print(len(following_limit))

7


# Extracción de la información de las cuentas Followers

In [12]:
# Existe limitación al traer la información de los seguidores
data = []
for followers in followers_limit:
    follower_obj=api.lookup_users(user_ids=followers) 
    for user in follower_obj:
        follower={}
        follower["id"]=user.id
        follower["name"]=user.name
        follower["screen_name"]=user.screen_name
        follower["created_at"]=user.created_at
        follower["location"]=user.location
        follower["followers_count"]=user.followers_count
        follower["friends_count"]=user.friends_count
        follower["description"]=user.description
        data.append(follower)

df_followers_info = pd.DataFrame(data)
df_followers_info

NameError: name 'followers_limit' is not defined

# Definición de tramos de las cuentas Followings

In [13]:
# Existe limitación al traer la información de los seguidores
data = []
for followings in following_limit:
    following_obj=api.lookup_users(user_ids=followings) 
    for user in following_obj:
        following={}
        following["id"]=user.id
        following["name"]=user.name
        following["screen_name"]=user.screen_name
        following["created_at"]=user.created_at
        following["location"]=user.location
        following["followers_count"]=user.followers_count
        following["friends_count"]=user.friends_count
        following["description"]=user.description
        data.append(following)

df_following_info = pd.DataFrame(data)
df_following_info

,id,name,screen_name,created_at,location,followers_count,friends_count,description
0,714366811,Deborah Li,li_deb,2012-07-24 14:20:57,"Madrid, Spain",772,518,Expanding Europe-based #startups international...
1,39787443,carlos herrera,cyague,2009-05-13 16:41:20,"Madrid, Spain",420,255,"'%s/data/knowledge/g' networks, machine learli..."
2,194161870,Mar Gonzalez-Franco,twi_mar,2010-09-23 15:41:02,Seattle / Etxauri,3506,1815,Advancing the Technological Frontier from SW/H...
3,48422383,Jonathan Chacón,jonathanchacon,2009-06-18 17:29:37,Transitando por el 🌍,2482,214,Senior accessibility software engineer at @Cab...
4,1173984355800293377,Mobilidade Coruñesa,MobilidadeC,2019-09-17 15:37:47,,1,6,"Ideas, proxectos, actualidade e noticias sobre..."
5,2992924523,Cabify,cabify,2015-01-22 20:30:57,Global,2203,26,¡Bienvenido a Cabify! Si tienes cualquier duda...
6,715535722086801408,Mutua Madrileña,la_Mutua,2016-03-31 13:46:24,,5018,324,#JuntosEvolucionamos hacia todo lo que está po...
7,906103502841290752,Empresas por la Movilidad Sostenible,PlataformaEMS,2017-09-08 10:34:44,España,620,417,¡Impulsando la #MovilidadSostenible desde las ...
8,428136372,Luis Benavides,LluisBenavides,2011-12-04 11:35:52,"Barcelona, Spain",882,1685,El Periódico de Catalunya. Cuenta personal
9,913184120100081664,La Nave,LaNaveMadrid,2017-09-27 23:30:35,"Madrid, Spain",4821,912,"#LaNave, un espacio de innovación urbana para ..."


In [14]:
#Guardado en ficheros de los IDS de Twitter
def saveID(dataframe,file):
    dataframe.to_csv(file)

saveID(dataFollowing_id,'followings_id.csv')
#saveID(dataFollowers_id,'followers_id.csv')

In [15]:
#Guardado en ficheros de los IDS de Twitter
def saveInfo(dataframe,file):
    dataframe.to_csv(file)

saveInfo(df_following_info,'followings_info.csv')
#saveInfo(df_followers_info,'df_followers_info.csv')


In [65]:
for item in df_following_info["screen_name"]:
    print(str(item).replace(' ',''))

jonathanchacon
MobilidadeC
cabify
la_Mutua
MadridSummitMS
LluisBenavides
LaNaveMadrid
MadridMotion
CocaColaCo_es
marcvidal
unicomplutense
eBayESP
burn_es
VectorRonda
marta_plana
elpais_retina
NewEconomyForum
aeropuerto_LCG
WayraES
SemanaRSCMadrid
Ismarbar
AECID_es
davidperezpi
Isabelgarro
lycompany1
albertogil1980
ironhack
TetraPak_es
braysefe
IEbusiness
south_summit
IreneRecio
pabloviewpoint
DGTes
SEUMov
CarlosMarco
DanielGrao7
ivanferreiro
LopezAquinoL
ItziarCastro
yas_salem
esperanzagracia
MAPFRE
moniquecestmoi
unautovtc
movo_esp
PODEMOS
FAMMA_Co_Madrid
Toni_Rami
david_bel
borjaberzosa
carofigoli
jdalamo63
Goorgo
Nacho_Palomar
imartalamas
_PacoCampos
visualmethod
alexlamatrainer
AlbertoCastilla
Emma_Lebon
ANavarreteBueno
vegetta777
frnperea
Avaloncine
carlessanchez
Dahudi92
jcompan
erlantzplaza
luciabrotons
uyalbert
nspmediatv
Kobazuloak
MarcoGinja22
alivcamus
pedroorellanali
AlaineGonzalez
mariajo_suarez
modesto_garcia
Desaze
Wismichu
Joel_Fortuny
garciamadrid
albertochicote
grefus

# Extracción de la información de los Tweets de Followers

In [ ]:
#Extracción de los tweets de las cuentas de followers
data = []
for users in df_followers_info["screen_name"]:
    formato_user=str(users).replace(' ','')
    try:   
        tweets_user=api.user_timeline(screen_name=formato_user, count=1)
        for tweet in tweets_user:
            #Creación de diccionarios
            tweets={}
            tweets["id_tweet"]=tweet.id
            tweets["id_user"]=tweet.user.id
            tweets["text"]=tweet.text
            tweets["location"] = tweet.user.location
            tweets["created_at"]=tweet.created_at
            data.append(tweets)
    except tweepy.TweepError:
        print("Fallo por el usuario, ",tweet.user.id," Skipping...")
        pass
            
df_followers_info_tweets = pd.DataFrame(data)
df_followers_info_tweets

Fallo por el usuario,  85946106  Skipping...
Fallo por el usuario,  492595512  Skipping...
Fallo por el usuario,  1105167868621139968  Skipping...
Fallo por el usuario,  860100474715951104  Skipping...
Fallo por el usuario,  534771080  Skipping...
Fallo por el usuario,  1219916472086077440  Skipping...
Fallo por el usuario,  1076851198408491008  Skipping...
Fallo por el usuario,  1164276844398555136  Skipping...
Fallo por el usuario,  978689133009887232  Skipping...
Fallo por el usuario,  978689133009887232  Skipping...
Fallo por el usuario,  716574310236872704  Skipping...
Fallo por el usuario,  1221490818491539456  Skipping...
Fallo por el usuario,  77385383  Skipping...
Fallo por el usuario,  300416495  Skipping...
Fallo por el usuario,  3359130580  Skipping...
Fallo por el usuario,  1146332994346586112  Skipping...
Fallo por el usuario,  1068642743642177536  Skipping...
Fallo por el usuario,  1219382874459312130  Skipping...
Fallo por el usuario,  157800367  Skipping...
Fallo por e

Rate limit reached. Sleeping for: 551


Fallo por el usuario,  2473700540  Skipping...
Fallo por el usuario,  948094614447296512  Skipping...
Fallo por el usuario,  222880372  Skipping...
Fallo por el usuario,  47808922  Skipping...
Fallo por el usuario,  1349452261  Skipping...
Fallo por el usuario,  2550165192  Skipping...
Fallo por el usuario,  2550165192  Skipping...
Fallo por el usuario,  148545917  Skipping...
Fallo por el usuario,  3825432785  Skipping...
Fallo por el usuario,  175769602  Skipping...
Fallo por el usuario,  175769602  Skipping...
Fallo por el usuario,  940194606804819969  Skipping...
Fallo por el usuario,  16866767  Skipping...
Fallo por el usuario,  63854034  Skipping...
Fallo por el usuario,  490649122  Skipping...
Fallo por el usuario,  1206602409058979840  Skipping...
Fallo por el usuario,  361930079  Skipping...
Fallo por el usuario,  122479094  Skipping...
Fallo por el usuario,  743633306  Skipping...
Fallo por el usuario,  289014735  Skipping...
Fallo por el usuario,  407348888  Skipping...
Fall

Rate limit reached. Sleeping for: 570


Fallo por el usuario,  80122262  Skipping...
Fallo por el usuario,  1110541048424656897  Skipping...
Fallo por el usuario,  398076160  Skipping...
Fallo por el usuario,  1105580492663984129  Skipping...
Fallo por el usuario,  1105580492663984129  Skipping...
Fallo por el usuario,  1158390396688510976  Skipping...
Fallo por el usuario,  1159808156526141440  Skipping...
Fallo por el usuario,  1143911036896129029  Skipping...
Fallo por el usuario,  1143911036896129029  Skipping...
Fallo por el usuario,  264448270  Skipping...
Fallo por el usuario,  583920940  Skipping...
Fallo por el usuario,  2651450103  Skipping...
Fallo por el usuario,  1156806374132764672  Skipping...
Fallo por el usuario,  403012226  Skipping...
Fallo por el usuario,  47756149  Skipping...
Fallo por el usuario,  563864182  Skipping...
Fallo por el usuario,  468258123  Skipping...
Fallo por el usuario,  2166636186  Skipping...
Fallo por el usuario,  1319944850  Skipping...
Fallo por el usuario,  2600516221  Skipping..

...

Rate limit reached. Sleeping for: 579


Fallo por el usuario,  373075356  Skipping...
Fallo por el usuario,  45088970  Skipping...
Fallo por el usuario,  239378243  Skipping...
Fallo por el usuario,  563194546  Skipping...
Fallo por el usuario,  563194546  Skipping...
Fallo por el usuario,  137450386  Skipping...
Fallo por el usuario,  1003533942283296768  Skipping...
Fallo por el usuario,  416636571  Skipping...
Fallo por el usuario,  416636571  Skipping...
Fallo por el usuario,  2155044775  Skipping...
Fallo por el usuario,  2155044775  Skipping...
Fallo por el usuario,  2155044775  Skipping...
Fallo por el usuario,  164809876  Skipping...
Fallo por el usuario,  298420061  Skipping...
Fallo por el usuario,  8203462  Skipping...
Fallo por el usuario,  8203462  Skipping...
Fallo por el usuario,  411304022  Skipping...
Fallo por el usuario,  242962661  Skipping...
Fallo por el usuario,  557109228  Skipping...
Fallo por el usuario,  557109228  Skipping...
Fallo por el usuario,  987316429044232192  Skipping...
Fallo por el usua

Rate limit reached. Sleeping for: 572


Fallo por el usuario,  472327972  Skipping...
Fallo por el usuario,  126449833  Skipping...
Fallo por el usuario,  61569399  Skipping...
Fallo por el usuario,  375340011  Skipping...
Fallo por el usuario,  139000249  Skipping...
Fallo por el usuario,  139000249  Skipping...
Fallo por el usuario,  906710576  Skipping...
Fallo por el usuario,  353371761  Skipping...
Fallo por el usuario,  389397636  Skipping...
Fallo por el usuario,  1431594596  Skipping...
Fallo por el usuario,  56730456  Skipping...
Fallo por el usuario,  220958088  Skipping...
Fallo por el usuario,  99908204  Skipping...
Fallo por el usuario,  210132620  Skipping...
Fallo por el usuario,  734092114410262533  Skipping...
Fallo por el usuario,  2821116531  Skipping...
Fallo por el usuario,  993251414527938562  Skipping...
Fallo por el usuario,  1153668811  Skipping...
Fallo por el usuario,  287759778  Skipping...
Fallo por el usuario,  166160529  Skipping...
Fallo por el usuario,  166160529  Skipping...
Fallo por el usu

Rate limit reached. Sleeping for: 587


# Extracción de la información de los Tweets de Followings

In [22]:
#Extracción de los tweets de las cuentas de followers
data = []
for users in df_following_info["screen_name"]:
    formato_user=str(users).replace(' ','')
    try:   
        tweets_user=api.user_timeline(screen_name=formato_user, count=1)
        for tweet in tweets_user:
            #Creación de diccionarios
            tweets={}
            tweets["id_tweet"]=tweet.id
            tweets["id_user"]=tweet.user.id
            tweets["text"]=tweet.text
            tweets["location"] = tweet.user.location
            tweets["created_at"]=tweet.created_at
            data.append(tweets)
    except tweepy.TweepError:
        print("Fallo por el usuario, ",tweet.user.id," Skipping...")
        pass
            
df_following_info_tweets = pd.DataFrame(data)
df_following_info_tweets

Fallo por el usuario,  867615151951925248  Skipping...
Fallo por el usuario,  105895230  Skipping...
Fallo por el usuario,  816178  Skipping...
Fallo por el usuario,  169411324  Skipping...
Fallo por el usuario,  715656119839023104  Skipping...
Fallo por el usuario,  51736879  Skipping...
Fallo por el usuario,  145994453  Skipping...
Fallo por el usuario,  157705342  Skipping...
Fallo por el usuario,  316695810  Skipping...


,id_tweet,id_user,text,location,created_at
0,1225785867832381441,48422383,Some projects are about #accessibility 🙌 and I...,Transitando por el 🌍,2020-02-07 14:18:19
1,1225031619653795840,2992924523,Financial Times @FT entrevista a nuestro funda...,Global,2020-02-05 12:21:12
2,1225765733013098502,715535722086801408,RT @interbrandMAD: .@la_Mutua se mueve constan...,,2020-02-07 12:58:19
3,1222537192947535872,906103502841290752,RT @MadridSummitMS: ¡Esto es “Empresas por la ...,"Madrid, Comunidad de Madrid",2020-01-29 15:09:15
4,1225826091295813632,428136372,RT @martarias: Mañana sábado hay nueva edición...,"Barcelona, España",2020-02-07 16:58:09
5,1225816831367286784,913184120100081664,🤔 ¿Qué podemos hacer para tener un consumo de ...,"Madrid, España",2020-02-07 16:21:21
6,1225747481293709312,1076052488921460736,La nueva iniciativa en #movilidad sostenible e...,,2020-02-07 11:45:47
7,1225813993878671360,2848321193,"🗣️ @jdeelizalde: ""Tener muy claro el objetivo ...","Madrid, España",2020-02-07 16:10:05
8,1225799564235546624,9974322,📢Debería ser noticia 👇\n\nEl primer ensayo de ...,Madrid,2020-02-07 15:12:45
9,1225783557622243328,85343435,Tiempo de descanso al sol\n☀️\n☀️\n#estudiante...,#MuyCerca,2020-02-07 14:09:08


In [35]:
#Guardado en ficheros de los tweets por usuario de Twitter
def saveTweets(dataframe,file):
    dataframe.to_csv(file)

saveTweets(df_following_info_tweets,'followings_info_tweets.csv')
saveTweets(df_followers_info_tweets,'df_followers_info_tweets.csv')

NameError: name 'df_following_info_tweets' is not defined